# Module 4: GARCH Models and LSTM for Financial Time Series

**CRITICAL FOR QUANTITATIVE FINANCE**

This notebook covers volatility modeling with GARCH and deep learning with LSTM for financial forecasting.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("GARCH and LSTM modules loaded!")

## 1. GARCH Models for Volatility

**GARCH(p,q)**: Generalized Autoregressive Conditional Heteroskedasticity

$$\sigma_t^2 = \omega + \sum_{i=1}^{p} \alpha_i \epsilon_{t-i}^2 + \sum_{j=1}^{q} \beta_j \sigma_{t-j}^2$$

**GARCH(1,1)** is most common:
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

**Properties:**
- Models volatility clustering
- Captures leverage effects with GJR-GARCH
- Essential for option pricing and risk management

In [ ]:
# Download financial data
ticker = 'SPY'
start_date = '2015-01-01'
end_date = '2024-11-01'

print(f"Downloading {ticker} data...")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)
prices = data['Close']
returns = 100 * prices.pct_change().dropna()  # Returns in percentage

print(f"\nData Summary:")
print(f"  Date Range: {returns.index[0]} to {returns.index[-1]}")
print(f"  Observations: {len(returns)}")
print(f"\nReturns Statistics:")
print(returns.describe())

# Plot returns
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

ax1.plot(prices.index, prices.values, linewidth=1)
ax1.set_title(f'{ticker} Price', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.grid(True, alpha=0.3)

ax2.plot(returns.index, returns.values, linewidth=0.5, alpha=0.7)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax2.set_title(f'{ticker} Returns (%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Returns (%)', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Fit GARCH(1,1) Model

In [ ]:
# Fit GARCH(1,1) model
print("\nFitting GARCH(1,1) model...")

# Using ARCH package best practices
model_garch = arch_model(returns, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
res_garch = model_garch.fit(disp='off')

print("\n" + "="*70)
print("GARCH(1,1) ESTIMATION RESULTS")
print("="*70)
print(res_garch.summary())

# Extract parameters
omega = res_garch.params['omega']
alpha = res_garch.params['alpha[1]']
beta = res_garch.params['beta[1]']

print(f"\nKey Parameters:")
print(f"  ω (omega): {omega:.6f}")
print(f"  α (alpha): {alpha:.6f}")
print(f"  β (beta):  {beta:.6f}")
print(f"  α + β:     {alpha + beta:.6f}  (< 1 for stationarity)")

# Persistence
persistence = alpha + beta
print(f"\nVolatility Persistence: {persistence:.6f}")
print(f"Half-life of shocks: {np.log(0.5) / np.log(persistence):.2f} days")

In [ ]:
# Extract conditional volatility
cond_vol = res_garch.conditional_volatility

# Annualized volatility
ann_vol = cond_vol * np.sqrt(252)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Returns and conditional volatility
ax = axes[0]
ax.plot(returns.index, returns.values, linewidth=0.5, alpha=0.5, label='Returns')
ax.plot(cond_vol.index, cond_vol.values, 'r-', linewidth=2, label='Conditional Volatility')
ax.plot(cond_vol.index, -cond_vol.values, 'r-', linewidth=2)
ax.fill_between(cond_vol.index, -cond_vol.values, cond_vol.values, alpha=0.2, color='red')
ax.set_title('Returns with GARCH(1,1) Conditional Volatility', fontsize=14, fontweight='bold')
ax.set_ylabel('Returns (%)', fontsize=12)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Annualized volatility
ax = axes[1]
ax.plot(ann_vol.index, ann_vol.values, linewidth=2, color='darkblue')
ax.axhline(y=ann_vol.mean(), color='red', linestyle='--', linewidth=2, 
          label=f'Mean: {ann_vol.mean():.2f}%')
ax.set_title('Annualized Volatility (GARCH)', fontsize=14, fontweight='bold')
ax.set_ylabel('Volatility (%)', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAnnualized Volatility Statistics:")
print(f"  Mean: {ann_vol.mean():.2f}%")
print(f"  Min:  {ann_vol.min():.2f}%")
print(f"  Max:  {ann_vol.max():.2f}%")

## 3. GJR-GARCH for Asymmetric Effects

**GJR-GARCH** captures leverage effect (negative shocks increase volatility more):

$$\sigma_t^2 = \omega + (\alpha + \gamma I_{t-1})\epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

Where $I_{t-1} = 1$ if $\epsilon_{t-1} < 0$, else 0.

In [ ]:
# Fit GJR-GARCH
print("\nFitting GJR-GARCH(1,1) model...")

model_gjr = arch_model(returns, vol='Garch', p=1, o=1, q=1, mean='Constant', dist='normal')
res_gjr = model_gjr.fit(disp='off')

print("\n" + "="*70)
print("GJR-GARCH(1,1) ESTIMATION RESULTS")
print("="*70)
print(res_gjr.summary())

# Extract gamma (asymmetry parameter)
gamma = res_gjr.params['gamma[1]']
print(f"\nAsymmetry Parameter (γ): {gamma:.6f}")
print(f"Interpretation: {'Leverage effect present' if gamma > 0 else 'No leverage effect'}")

# Compare models using AIC/BIC
print(f"\nModel Comparison:")
print(f"  GARCH(1,1)     AIC: {res_garch.aic:.2f}  BIC: {res_garch.bic:.2f}")
print(f"  GJR-GARCH(1,1) AIC: {res_gjr.aic:.2f}  BIC: {res_gjr.bic:.2f}")
print(f"  Better model: {'GJR-GARCH' if res_gjr.aic < res_garch.aic else 'GARCH'}")

## 4. Volatility Forecasting with GARCH

In [ ]:
# Forecast volatility
forecast_horizon = 30  # days

print(f"\nForecasting volatility for next {forecast_horizon} days...")
forecasts = res_garch.forecast(horizon=forecast_horizon, reindex=False)

# Extract forecast variance
forecast_variance = forecasts.variance.values[-1, :]
forecast_vol = np.sqrt(forecast_variance)
forecast_ann_vol = forecast_vol * np.sqrt(252)

# Plot forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Historical volatility
hist_vol = ann_vol.iloc[-100:]  # Last 100 days
ax.plot(hist_vol.index, hist_vol.values, 'b-', linewidth=2, label='Historical Volatility')

# Forecast
last_date = hist_vol.index[-1]
forecast_dates = pd.date_range(start=last_date, periods=forecast_horizon+1, freq='B')[1:]
ax.plot(forecast_dates, forecast_ann_vol, 'r--', linewidth=2, label='GARCH Forecast')

# Add confidence interval (approximate)
std_error = forecast_ann_vol * 0.1  # Simplified
ax.fill_between(forecast_dates, 
               forecast_ann_vol - 1.96*std_error,
               forecast_ann_vol + 1.96*std_error,
               alpha=0.2, color='red', label='95% CI')

ax.set_title(f'{ticker} Volatility Forecast (GARCH)', fontsize=14, fontweight='bold')
ax.set_ylabel('Annualized Volatility (%)', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n{forecast_horizon}-Day Forecast:")
print(f"  Day 1 volatility:  {forecast_ann_vol[0]:.2f}%")
print(f"  Day {forecast_horizon} volatility: {forecast_ann_vol[-1]:.2f}%")
print(f"  Mean forecast:     {forecast_ann_vol.mean():.2f}%")

## 5. LSTM for Return Prediction

**Long Short-Term Memory (LSTM)** networks can capture complex temporal dependencies.

In [ ]:
# Check if TensorFlow is available
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    
    tf_available = True
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    tf_available = False
    print("TensorFlow not installed. Install with: pip install tensorflow")

if tf_available:
    # Prepare data for LSTM
    def create_sequences(data, seq_length=60):
        """
        Create sequences for LSTM.
        
        Parameters:
        -----------
        data : array
            Time series data
        seq_length : int
            Sequence length (lookback period)
        
        Returns:
        --------
        X, y : arrays
            Features and targets
        """
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:i+seq_length])
            y.append(data[i+seq_length])
        return np.array(X), np.array(y)
    
    # Prepare returns data
    returns_array = returns.values.reshape(-1, 1)
    
    # Scale data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    returns_scaled = scaler.fit_transform(returns_array)
    
    # Create sequences
    seq_length = 60  # Use 60 days to predict next day
    X, y = create_sequences(returns_scaled, seq_length)
    
    # Train/test split (80/20)
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    print(f"\nLSTM Data Preparation:")
    print(f"  Sequence length: {seq_length} days")
    print(f"  Training samples: {len(X_train)}")
    print(f"  Testing samples: {len(X_test)}")
    print(f"  Input shape: {X_train.shape}")
    
    # Build LSTM model
    print("\nBuilding LSTM model...")
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print(model.summary())
    
    # Train model
    print("\nTraining LSTM (this may take a few minutes)...")
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=50,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=0
    )
    
    print("\nTraining complete!")
    
    # Make predictions
    y_pred_train = model.predict(X_train, verbose=0)
    y_pred_test = model.predict(X_test, verbose=0)
    
    # Inverse transform
    y_train_orig = scaler.inverse_transform(y_train.reshape(-1, 1))
    y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_train_orig = scaler.inverse_transform(y_pred_train)
    y_pred_test_orig = scaler.inverse_transform(y_pred_test)
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train_orig, y_pred_train_orig)
    test_mse = mean_squared_error(y_test_orig, y_pred_test_orig)
    train_mae = mean_absolute_error(y_train_orig, y_pred_train_orig)
    test_mae = mean_absolute_error(y_test_orig, y_pred_test_orig)
    
    print(f"\nLSTM Performance:")
    print(f"  Train MSE: {train_mse:.6f}")
    print(f"  Test MSE:  {test_mse:.6f}")
    print(f"  Train MAE: {train_mae:.6f}")
    print(f"  Test MAE:  {test_mae:.6f}")
    
    # Plot results
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Training history
    ax = axes[0]
    ax.plot(history.history['loss'], label='Training Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_title('LSTM Training History', fontsize=14, fontweight='bold')
    ax.set_ylabel('Loss (MSE)', fontsize=12)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Predictions vs Actual (test set)
    ax = axes[1]
    test_dates = returns.index[split_idx+seq_length:]
    ax.plot(test_dates, y_test_orig, label='Actual Returns', alpha=0.7, linewidth=1)
    ax.plot(test_dates, y_pred_test_orig, label='LSTM Predictions', alpha=0.7, linewidth=1)
    ax.set_title('LSTM Predictions vs Actual Returns (Test Set)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Returns (%)', fontsize=12)
    ax.set_xlabel('Date', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("\nSkipping LSTM example. Install TensorFlow to run this section.")
    print("Command: pip install tensorflow")

## Key Takeaways

### GARCH Models
1. **Volatility Clustering**: GARCH captures periods of high/low volatility
2. **Persistence**: α + β measures how long shocks persist
3. **Leverage Effect**: GJR-GARCH models asymmetric volatility response
4. **Forecasting**: GARCH provides conditional volatility forecasts
5. **Applications**: Option pricing, VaR, risk management

### LSTM Networks
1. **Sequence Learning**: Captures long-term dependencies
2. **Nonlinear Patterns**: Can model complex relationships
3. **Feature Engineering**: Minimal manual feature extraction needed
4. **Overfitting**: Requires careful regularization (dropout, early stopping)
5. **Limitations**: Not always better than simpler models; hard to interpret

### Practical Insights
- **GARCH**: Best for volatility forecasting and risk metrics
- **LSTM**: Experimental for return prediction; use with caution
- **Hybrid**: Combine GARCH volatility with LSTM for features
- **Validation**: Always backtest on out-of-sample data
- **Production**: GARCH more reliable for production systems

**CAREER RELEVANCE:**
- GARCH knowledge essential for risk management roles
- LSTM shows ML/AI skills valued by quant firms
- Understanding both demonstrates versatility

## 📝 Guided Exercises with Auto-Validation

Practice volatility modeling and understand GARCH parameters!

### Exercise 1: GARCH Persistence and Half-Life (Intermediate)

Calculate volatility persistence from GARCH parameters.

In [ ]:
# Exercise 1: GARCH Persistence
import numpy as np

# Given GARCH(1,1) parameters from estimation
omega = 0.00001    # Constant term
alpha = 0.08       # ARCH coefficient (reaction to shocks)
beta = 0.90        # GARCH coefficient (persistence)

# TODO: Calculate persistence (α + β)
persistence = None

# TODO: Calculate half-life of volatility shocks
# Formula: half_life = ln(0.5) / ln(persistence)
half_life_days = None

# TODO: Check for stationarity (persistence < 1)
is_stationary = None  # True or False

# ============= AUTO-VALIDATION (DO NOT MODIFY) =============
assert persistence is not None, "❌ Calculate persistence!"
assert half_life_days is not None, "❌ Calculate half-life!"
assert is_stationary is not None, "❌ Check stationarity!"
assert persistence == alpha + beta, f"❌ Persistence should be α + β = {alpha + beta}"
expected_half_life = np.log(0.5) / np.log(persistence)
assert np.isclose(half_life_days, expected_half_life, rtol=0.01), f"❌ Half-life incorrect. Expected {expected_half_life:.2f}, got {half_life_days:.2f}"
assert is_stationary == (persistence < 1), "❌ Stationarity check incorrect"
print("✅ Exercise 1 Complete!")
print(f"   Persistence (α + β): {persistence:.4f}")
print(f"   Half-life: {half_life_days:.2f} trading days")
print(f"   Stationary: {is_stationary}")
print(f"   Interpretation: Volatility shocks decay by half in ~{half_life_days:.0f} days")
# =========================================================